# Multivariate EBNM based prior for M&M

Here for the simulation benchmark we prepare mixture prior based on a mulrivariate Emperical Bayes Normal Mean model (previously we use Extreme Deconvolution for the task).

## Approach

Here is the analysis plan:

1. Identify up to 20K genes where there is complete phenotype data to make a good / realistic residual variance estimate via FLASH
2. Simulate 20K data under my phenotypic models (the latest DSC benchmark setting) and generate sumstats for them ; bhat and sbhat
3. For each data-set, take the strongest gene-snp pair as the strong set
4. Also select from each data-set perhaps 4 "random" gene-snp pair.
5. then try to run your estimate of Vhat to get Vhat first, and run Yunqi / Peter's ED

In GTEx we have >35K genes. The reason we want to try using 20K is that 20K seems to have enough information learning about the pattern of sharing between conditions. 

But we "cheat" a bit by simulating under identity residual variance for all genes, and fit EBNM assuming residual variance is identity, too; or just estimating a global residual variance. This makes the problem easier. Because in practice residual can be different (though maybe similar!) for different genes.

So the simplified plan is to only do 2~5 with 2 using just identity matrix for residual variance.

## Workflow

In [ ]:
[global]
parameter: cwd = path('/project2/mstephens/gaow/mvarbvs/dsc/mnm_prototype/mnm_sumstats')
parameter: model = 'artificial_mixture_identity' # 'gtex_mixture_identity'
parameter: per_chunk = 200
import glob

In [1]:
%cd /project2/mstephens/gaow/mvarbvs/dsc/mnm_prototype/mnm_sumstats

/project2/mstephens/gaow/mvarbvs/dsc/mnm_prototype/mnm_sumstats

### Get top gene-SNP and random gene-SNP pairs per gene

In [31]:
# extract data for MAHS from summary stats
[extract_1]
parameter: seed = 999
parameter: n_random = 4
input: glob.glob(f'{cwd}/{model}/*.rds'), group_by = per_chunk
output: f"{cwd}/{model}/cache/{model}_{_index+1}.rds"
task: trunk_workers = 1, walltime = '1h', trunk_size = 1, mem = '4G', cores = 1, tags = f'{_output:bn}'
R: expand = "${ }"
    set.seed(${seed})
    matxMax <- function(mtx) {
        max_idx <- which.max(mtx)
        colmn <- max_idx %/% nrow(mtx) + 1
        row <- max_idx %% nrow(mtx)
        return( matrix(c(row, colmn), 1))
    }
    remove_rownames = function(x) {
        for (name in names(x)) rownames(x[[name]]) = NULL
        return(x)
    }
    extract_one_data = function(infile, n_random) {
        # If cannot read the input for some reason then let it go. I dont care losing one.
        dat = tryCatch(readRDS(infile)$sumstats, error = function(e) return(NULL))
        if (is.null(dat)) return(NULL)
        z = abs(dat$bhat/dat$sbhat)
        max_idx = matxMax(z)
        strong = list(bhat = dat$bhat[max_idx[1],,drop=F], sbhat = dat$sbhat[max_idx[1],,drop=F])
        if (max_idx[1] == 1) {
            sample_idx = 2:nrow(z)
        } else if (max_idx[1] == nrow(z)) {
            sample_idx = 1:(max_idx[1]-1)
        } else {
            sample_idx = c(1:(max_idx[1]-1), (max_idx[1]+1):nrow(z))
        }
        random_idx = sample(sample_idx, n_random, replace = T)
        random = list(bhat = dat$bhat[random_idx,,drop=F], sbhat = dat$sbhat[random_idx,,drop=F])
        return(list(random = remove_rownames(random),  strong = remove_rownames(strong)))
    }
    merge_data = function(res, one_data) {
      if (length(res) == 0) {
          return(one_data)
      } else if (is.null(one_data)) {
          return(res)
      } else {
          for (d in names(one_data)) {
              for (s in names(one_data[[d]])) {
                  res[[d]][[s]] = rbind(res[[d]][[s]], one_data[[d]][[s]])
              }
          }
          return(res)
      }
    }
    res = list()
    for (f in c(${_input:r,})) {
      res = merge_data(res, extract_one_data(f, ${n_random}))
    }
    saveRDS(res, ${_output:r})
  
[extract_2]
input: group_by = "all"
output: f"{cwd}/{model}.rds"
task: trunk_workers = 1, walltime = '1h', trunk_size = 1, mem = '4G', cores = 1, tags = f'{_output:bn}'
R: expand = "${ }"
    merge_data = function(res, one_data) {
      if (length(res) == 0) {
          return(one_data)
      } else {
          for (d in names(one_data)) {
              for (s in names(one_data[[d]])) {
                  res[[d]][[s]] = rbind(res[[d]][[s]], one_data[[d]][[s]])
              }
          }
          return(res)
      }
    }
    dat = list()
    for (f in c(${_input:r,})) {
      dat = merge_data(dat, readRDS(f))
    }
    # make output consistent in format with 
    # https://github.com/stephenslab/gtexresults/blob/master/workflows/mashr_flashr_workflow.ipynb
    saveRDS(
          list(random.z = dat$random$bhat/dat$random$sbhat,
           strong.z = dat$strong$bhat/dat$strong$sbhat, 
           random.b = dat$random$bhat,
           strong.b = dat$strong$bhat,
           random.s = dat$random$sbhat,
           strong.s = dat$strong$sbhat),
          ${_output:r})

To run it:

```
for m in artificial_mixture_identity gtex_mixture_identity; do 
    sos run analysis/20200502_Prepare_ED_prior.ipynb extract --model $m -c midway2.yml -q midway2
done
```

## Run extreme deconvolution using `mashr`

Before this, we need to run the following to generate FLASH mixture,

```
for m in artificial_mixture_identity gtex_mixture_identity; do
sos run ~/GIT/gtexresults/workflows/mashr_flashr_workflow.ipynb flash \
    --cwd /project2/mstephens/gaow/mvarbvs/dsc/mnm_prototype/mnm_sumstats/ \
    --data /project2/mstephens/gaow/mvarbvs/dsc/mnm_prototype/mnm_sumstats/$m.rds \
    --effect-model EE
done
```

We will use `simple` method to compute the residual variance, as implemented in the pipeline below.

In [ ]:
[ed_1, teem_1]
depends: R_library("mashr")
parameter: npc = 3
input: f"{cwd}/{model}.rds", f"{cwd}/{model}.EE.flash.rds"
output: f"{cwd}/{model}.FLASH_PC{npc}.rds"
R: expand = "${ }", workdir = cwd, stderr = f"{_output:n}.stderr", stdout = f"{_output:n}.stdout"
    library(mashr)
    dat = readRDS(${_input[0]:r})
    dat$random.s[which(dat$random.b == 0 & dat$random.s == 0)] = 1E3
    vhat = estimate_null_correlation_simple(mash_set_data(dat$random.b, Shat=dat$random.s))
    mash_data = mash_set_data(dat$strong.b, Shat=dat$strong.s, alpha=0, V=vhat)
    # FLASH matrices
    U.flash = readRDS(${_input[1]:r})
    # SVD matrices
    U.pca = ${"cov_pca(mash_data, %s)" % npc if npc > 0 else "list()"}
    # Emperical cov matrix
    X.center = apply(mash_data$Bhat, 2, function(x) x - mean(x))
    Ulist = c(U.flash, U.pca, list("XX" = t(X.center) %*% X.center / nrow(X.center)))
    saveRDS(list(mash_data = mash_data, Ulist = Ulist), ${_output:r})

In [ ]:
[ed_2]
output: f"{_input:n}.ED.rds"
task: trunk_workers = 1, walltime = '36h', trunk_size = 1, mem = '4G', cores = 4, tags = f'{_output:bn}'
R: expand = "${ }", workdir = cwd, stderr = f"{_output:n}.stderr", stdout = f"{_output:n}.stdout"
    library(mashr)
    dat = readRDS(${_input:r})
    # Denoised data-driven matrices
    res = bovy_wrapper(dat$mash_data, dat$Ulist, logfile=${_output:nr}, tol = 5e-06)
    # format to input for simulation with DSC (current pipeline)
    saveRDS(list(U=res$Ulist, w=res$pi, loglik=scan("${_output:n}.ED_loglike.log")), ${_output:r}) 

In [ ]:
[teem_2]
output: f"{_input:n}.TEEM.rds"
task: trunk_workers = 1, walltime = '1h', trunk_size = 1, mem = '4G', cores = 1, tags = f'{_output:bn}'
R: expand = "${ }", workdir = cwd, stderr = f"{_output:n}.stderr", stdout = f"{_output:n}.stdout"
    library(mashr)
    dat = readRDS(${_input:r})
    # Denoised data-driven matrices
    res = teem_wrapper(dat$mash_data, dat$Ulist)
    saveRDS(res, ${_output:r})   

```
sos run analysis/20200502_Prepare_ED_prior.ipynb ed --model artificial_mixture_identity -c midway2.yml -q midway2
sos run analysis/20200502_Prepare_ED_prior.ipynb ed --model gtex_mixture_identity -c midway2.yml -q midway2
sos run analysis/20200502_Prepare_ED_prior.ipynb teem --model artificial_mixture_identity -c midway2.yml -q midway2
sos run analysis/20200502_Prepare_ED_prior.ipynb teem --model gtex_mixture_identity -c midway2.yml -q midway2
```

It takes many hours to run ED (almost a day) but only a few minutes to run TEEM. In fact TEEM on `gtex_mixture` based data converged in a second.

In [1]:
%cd ~/tmp/07-May-2020

/home/gw/tmp/07-May-2020

## Data-driven prior via ED

For the artifically simulated mixture,

In [2]:
a1 = readRDS('artificial_mixture_identity.FLASH_PC3.ED.rds')

In [3]:
names(a1)

[1] "U"      "w"      "loglik"

In [4]:
a1$loglik[length(a1$loglik)]

[1] 95.40111

In [5]:
a1$w

[1] 0.000000000 0.000000000 0.000000000 0.000000000 0.000000000 0.898314146
 [7] 0.000000000 0.000000000 0.000648316 0.063706371 0.037331167

In [6]:
names(a1$U)[which(a1$w>0.0001)]

[1] "tFLASH" "PCA_3"  "tPCA"   "XX"

The component with strongest weight is `tFLASH`. `tPCA` and the empirical covariance (`XX`) also have some weights.

For mixture simulated based on GTEx V8 ED matrices,

In [7]:
g1 = readRDS('gtex_mixture_identity.FLASH_PC3.ED.rds')

In [8]:
g1$loglik[length(g1$loglik)]

[1] 109.6084

In [9]:
g1$w

[1] 4.201670e-10 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 [6] 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
[11] 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
[16] 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 3.295366e-02
[21] 9.410475e-06 0.000000e+00 0.000000e+00 9.669866e-01 5.036723e-05

In [10]:
names(g1$U)[which(g1$w>0.0001)]

[1] "tFLASH" "tPCA"

Again, most weights are on `tFLASH` and `tPCA`. Only this time `tPCA` seems to have more weights than `tFLASH`.

## Data-driven prior via TEEM

Currently two caveats: 

1. we don't have interface for bhat/sbhat so the result is based on z-score without transforming back to original scale bhat.
2. we don't have interface for non-identity residual covariance so the result is based on assuming identity covariance, although identity covariance is indeed the oracle covariance for the residual.

In [11]:
a2 = readRDS('artificial_mixture_identity.FLASH_PC3.TEEM.rds')

In [12]:
names(a2)

[1] "w"         "U"         "objective" "maxd"

In [13]:
a2$objective[length(a2$objective)]

[1] -1514460

In [14]:
a2$w

[1] 1.501686e-04 6.370631e-02 1.739402e-03 2.256258e-05 2.744157e-05
 [6] 4.013501e-04 1.000047e-04 1.762108e-02 5.943179e-01 3.214616e-01
[11] 4.521734e-04

In [15]:
names(a2$U)[which(a2$w>0.0001)]

[1] "FLASH_1" "FLASH_2" "FLASH_3" "tFLASH"  "PCA_1"   "PCA_2"   "PCA_3"  
[8] "tPCA"    "XX"

**Different from ED results, most weights here are on `PCA_3` and `tPCA`, and not `tFLASH` and `XX`.**

In [16]:
g2 = readRDS('gtex_mixture_identity.FLASH_PC3.TEEM.rds')

In [17]:
g2$objective[length(g2$objective)]

[1] -1525156

In [18]:
g2$w

[1]  1.719557e-63 1.397773e-143 1.688457e-142  7.846913e-93 3.244862e-107
 [6] 1.314552e-135 1.336535e-138  5.878649e-88 1.577159e-134 2.621484e-134
[11] 1.983842e-132 1.195918e-134 1.096910e-133 2.116053e-141 6.153096e-125
[16] 2.049477e-139 1.679131e-114 5.631910e-123 5.662857e-139  8.034692e-87
[21]  1.687193e-63 1.742688e-146 1.642108e-146  1.000000e+00  9.352107e-87

In [19]:
names(g2$U)[which(g2$w>0.0001)]

[1] "tPCA"

**In ED result, `tPCA` gets 96% weights and `tFLASH` 3%. Here `tPCA` gets 100% weights.**